## To do:

* Extract names(nouns), places, countries
* EDA
* Sentiment analysis --> PCA? SVD? CorEx?
* Modeling

In [1]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [12]:
ls ../..

Invalid switch - "..".


In [2]:
# read data
data = pd.read_csv("../../data/steam_reviews-ENGLISH-sample.csv")

In [3]:
data.head()

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
1,292030,The Witcher 3: Wild Hunt,85184171,english,"good story, good graphics. lots to do.",1611379264,1611379264,True,0,0,...,True,False,False,76561198170193529,11,1,823.0,823.0,823.0,1.611379e+09
2,292030,The Witcher 3: Wild Hunt,85184064,english,"dis gud,",1611379091,1611379091,True,0,0,...,True,False,False,76561198119302812,27,2,4192.0,3398.0,4192.0,1.611352e+09
3,292030,The Witcher 3: Wild Hunt,85180436,english,favorite game of all time cant wait for the Ne...,1611373086,1611373086,True,0,0,...,True,False,False,76561198065591528,33,1,23329.0,177.0,23329.0,1.611219e+09
4,292030,The Witcher 3: Wild Hunt,85179753,english,Why wouldn't you get this,1611371978,1611371978,True,0,0,...,True,False,False,76561198996835044,131,2,8557.0,2004.0,8557.0,1.611371e+09


In [4]:
data.shape

(100000, 22)

In [5]:
# function developed by Rafał Wójcik from https://towardsdatascience.com/unsupervised-sentiment-analysis-a38bf1906483
# with some minor changes

def clean_text(text):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\?", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"[0-9]", " ", text)


    
    # text = text.split()

    return text


In [6]:
data['review'] = data['review'].apply(lambda x: clean_text(x))

In [7]:
data['review']

0        one of the best rpg's of all time worthy of an...
1                     good story good graphics lots to do 
2                                                 dis gud 
3        favorite game of all time cant wait for the ne...
4                                why wouldn't you get this
                               ...                        
99995                                     pretty good game
99996    the greatest game ever made if you haven't pla...
99997                      amazing graphics and storyline 
99998    my only regret is that i will never experience...
99999    i definitely recommend this game this game it ...
Name: review, Length: 100000, dtype: object

In [8]:
# dealing with words with too many characters
data['review'] = data['review'].apply(lambda x: [i if len(i)<27 else np.nan  for i in x])

In [9]:
data['review'] = data['review'].apply(lambda x: ''.join(x))

In [10]:
data['review'][0]

"one of the best rpg's of all time worthy of any collection"

In [11]:
data

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85184605,english,one of the best rpg's of all time worthy of an...,1611379970,1611379970,True,0,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
1,292030,The Witcher 3: Wild Hunt,85184171,english,good story good graphics lots to do,1611379264,1611379264,True,0,0,...,True,False,False,76561198170193529,11,1,823.0,823.0,823.0,1.611379e+09
2,292030,The Witcher 3: Wild Hunt,85184064,english,dis gud,1611379091,1611379091,True,0,0,...,True,False,False,76561198119302812,27,2,4192.0,3398.0,4192.0,1.611352e+09
3,292030,The Witcher 3: Wild Hunt,85180436,english,favorite game of all time cant wait for the ne...,1611373086,1611373086,True,0,0,...,True,False,False,76561198065591528,33,1,23329.0,177.0,23329.0,1.611219e+09
4,292030,The Witcher 3: Wild Hunt,85179753,english,why wouldn't you get this,1611371978,1611371978,True,0,0,...,True,False,False,76561198996835044,131,2,8557.0,2004.0,8557.0,1.611371e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,292030,The Witcher 3: Wild Hunt,37737848,english,pretty good game,1511746432,1511746432,True,0,0,...,False,False,False,76561198248862428,214,24,4263.0,0.0,1180.0,1.580618e+09
99996,292030,The Witcher 3: Wild Hunt,37737492,english,the greatest game ever made if you haven't pla...,1511746089,1511746089,True,1,0,...,True,False,False,76561198155221866,145,8,9971.0,0.0,9584.0,1.493935e+09
99997,292030,The Witcher 3: Wild Hunt,37737280,english,amazing graphics and storyline,1511745873,1511745873,True,0,0,...,True,False,False,76561198057394660,97,2,724.0,0.0,87.0,1.585956e+09
99998,292030,The Witcher 3: Wild Hunt,37737060,english,my only regret is that i will never experience...,1511745624,1516111635,True,0,0,...,True,False,False,76561198061575266,133,5,9313.0,0.0,8330.0,1.570408e+09


In [12]:
pwd

'c:\\Users\\fahadd\\T5_Bootcamp_directory\\PROJECTS\\game-reviews-sentiment-analysis\\workspace\\notebooks'

In [13]:
data.to_csv("../../data/steam_reviews-ENGLISH-sample-preprocessed.csv", index=False)